In [4]:
!pip install transformers datasets torchvision scikit-learn pandas matplotlib --quiet



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [5]:
from transformers import AutoFeatureExtractor, AutoModelForImageClassification
from torchvision import transforms
from PIL import Image
import torch
import os
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import pandas as pd


/Users/aditya/Desktop/GITHUB REPO/Predictive-Analytics/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
model_names = [
    "nateraw/bert-base-uncased-emotion", 
    "microsoft/beit-base-patch16-224",
    "facebook/deit-base-patch16-224",
    "google/vit-base-patch16-224",
    "microsoft/resnet-50",
    "microsoft/swin-base-patch4-window7-224",
    "hustvl/yolos-small",
    "nvidia/mit-b0",
    "microsoft/convnext-base"
]


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

image_dir = "/path/to/images" 

def load_images(image_dir):
    images, labels = [], []
    for label, subdir in enumerate(['real', 'fake']):
        for file in os.listdir(os.path.join(image_dir, subdir)):
            img_path = os.path.join(image_dir, subdir, file)
            try:
                img = Image.open(img_path).convert("RGB")
                img_tensor = transform(img)
                images.append(img_tensor)
                labels.append(label)
            except Exception as e:
                print(f"Error loading {img_path}: {e}")
    return torch.stack(images), labels

images, true_labels = load_images(image_dir)


In [ ]:
results = []

for model_id in model_names:
    try:
        extractor = AutoFeatureExtractor.from_pretrained(model_id)
        model = AutoModelForImageClassification.from_pretrained(model_id)
        model.eval()

        predictions = []
        for img in images:
            with torch.no_grad():
                inputs = extractor(images=img.permute(1, 2, 0).numpy(), return_tensors="pt")
                outputs = model(**inputs)
                predicted = torch.argmax(outputs.logits, dim=1).item()
                predictions.append(predicted)

        acc = accuracy_score(true_labels, predictions)
        prec = precision_score(true_labels, predictions, zero_division=0)
        rec = recall_score(true_labels, predictions, zero_division=0)
        f1 = f1_score(true_labels, predictions, zero_division=0)

        tn, fp, fn, tp = confusion_matrix(true_labels, predictions).ravel()
        sensitivity = tp / (tp + fn) if (tp + fn) else 0
        specificity = tn / (tn + fp) if (tn + fp) else 0

        results.append([sensitivity, specificity, prec, rec, f1, acc])

        print(f"Evaluated {model_id}")

    except Exception as e:
        print(f"Skipping {model_id} due to error: {e}")
        results.append([None] * 6)


In [7]:
df = pd.DataFrame(results, columns=["Sensitivity", "Specificity", "Precision", "Recall", "F1", "Accuracy"])
df.insert(0, "Models", [f"M{i+1}" for i in range(len(model_names))])

# Simple Ensemble: average of all metrics
ensemble_row = df.iloc[:, 1:].mean().to_list()
df.loc[len(df)] = ["Ensemble"] + ensemble_row

df


NameError: name 'results' is not defined

In [ ]:
df.to_csv("deepfake_model_metrics.csv", index=False)